In [1]:
import numpy as np
import xarray as xr
import dask
import xrft
import xgcm
import xmitgcm
import scipy.interpolate as int
import os.path as op
from glob import glob
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import scipy

import re
import traceback



%matplotlib inline

In [2]:
ddir = '/swot/SUM01/LLC/llc_4320_agulhas/'
fname = op.join(ddir, 'llc_4320_agulhas.0000010368.nc')
gname = ddir + 'llc_4320_agulhas.01.nc'



xr.open_mfdataset opens a MultiFile dataset, as opposed to open_dataset
.chunk() forces the data into Dask chunks

In [3]:
#create dataset
ds_data = xr.open_dataset(fname, decode_cf=False, autoclose=True)
#ds = xr.open_mfdataset(all_files[:10], decode_cf=False, autoclose=True)


grid_ds = xr.open_dataset(ddir + 'llc_4320_agulhas_grid.nc').chunk()
#print(grid_ds)


xr.merge() stores grid variables and data variables together

In [4]:
ds = xr.merge([grid_ds.drop('face'), ds_data.drop('face')], )
ds = xmitgcm.mds_store._swap_dimensions(ds, geometry='sphericalpolar')
# hack to fix coordinates
coord_names = set(grid_ds.coords).intersection(set(ds))
ds = ds.set_coords(coord_names)
#ds

In [5]:
ds

<xarray.Dataset>
Dimensions:   (XC: 2160, XG: 2160, YC: 2160, YG: 2160, Z: 90, Zl: 90, Zp1: 91, Zu: 90, time: 1)
Coordinates:
  * YC        (YC) float32 -57.001 -56.99 -56.9789 -56.9678 -56.9567 ...
  * YG        (YG) float32 -57.0066 -56.9955 -56.9844 -56.9733 -56.9623 ...
  * XC        (XC) float32 -15.4896 -15.4688 -15.4479 -15.4271 -15.4062 ...
  * XG        (XG) float32 -15.5 -15.4792 -15.4583 -15.4375 -15.4167 ...
  * Zp1       (Zp1) float32 0.0 -1.0 -2.14 -3.44 -4.93 -6.63 -8.56 -10.76 ...
  * Z         (Z) float32 -0.5 -1.57 -2.79 -4.185 -5.78 -7.595 -9.66 -12.01 ...
  * Zl        (Zl) float32 0.0 -1.0 -2.14 -3.44 -4.93 -6.63 -8.56 -10.76 ...
  * Zu        (Zu) float32 -1.0 -2.14 -3.44 -4.93 -6.63 -8.56 -10.76 -13.26 ...
    rA        (YC, XC) float32 1.5528e+06 1.5528e+06 1.5528e+06 1.5528e+06 ...
    rAw       (YC, XG) float32 1.5528e+06 1.5528e+06 1.5528e+06 1.5528e+06 ...
    rAs       (YG, XC) float32 9.96921e+36 9.96921e+36 9.96921e+36 ...
    rAz       (YG, XG) float32 1

In [ ]:
#Test out some spectra to see what detrending does

x = np.linspace(0, 2*np.pi, 100)
f =  1 + x + np.cos(2*np.pi*x)
f_notrend = np.cos(2*np.pi*x)
test_data = xr.DataArray(f, dims=['x'], coords={'x': x} )
test_data_notrend = xr.DataArray(f_notrend, dims=['x'], coords={'x': x} )

test_data.plot()
test_data_notrend.plot()

# x = np.linspace(0, 2*np.pi, 100)
# f =  1 + x + np.cos(x * 5)
# f_notrend = np.cos(x * 5)
# test_data = xr.DataArray(f, dims=['x'], coords={'x': x} )
# test_data_notrend = xr.DataArray(f_notrend, dims=['x'], coords={'x': x} )

# test_data.plot()
# test_data_notrend.plot()

In [ ]:
test_transform = xrft.dft(test_data).sel(freq_x=slice(0, np.inf))
test_transform_notrend = xrft.dft(test_data_notrend).sel(freq_x = slice(0, np.inf))
plot_spectrum_loglog(test_transform)
#test_transform.plot()
#test_transform_notrend.plot()

#np.real((np.conj(test_transform)*test_transform)).plot()
np.real((np.conj(test_transform_notrend)*test_transform_notrend)).plot()

In [ ]:
#test_ps = xrft.power_spectrum(test_data).sel(freq_x=slice(0, np.inf))

def plot_spectrum_loglog(ps, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    ps.plot(ax=ax)
    #ax.set_xscale('log')
    ax.set_yscale('log')
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    
    ax.set_ylabel('%s' %vars_name)
    return ax
    
#plot_spectrum_loglog(test_ps)
#plt.ylim((1e-8, 10))

In [ ]:
# test_ps_detrended = xrft.power_spectrum(test_data, detrend='linear').sel(freq_x=slice(0, np.inf))
# plot_spectrum_loglog(test_ps_detrended)
# plt.ylim((1e-8, 10))


# test_ps_notrend = xrft.power_spectrum(test_data_notrend, detrend=False).sel(freq_x=slice(0, np.inf))
# plot_spectrum_loglog(test_ps_notrend)

In [ ]:
def degrees_to_km(array, lon_0=None):
    """
    Parameters
    ----------
    array : DataArray, must be sliced at a single 
    value of lat ('YC'). Must have 'XC' as a coordinate variable
    corresponding to degrees lon
    
    lon_0 : float; longitude from which distances are calculated
    
    
    Returns
    -------
    Same DataArray with coordinate 'XC' replaced by distance 
    from Western most longitude, measured in km

        
    """
    
    
    # Takes in DataArray as argument. Array must be sliced 
    # at a single 'YC' value. Returns DataArry with same variable values but 'XC'
    # coordinates replaced with 'km'. Lon_0 coordinate is mapped to 0 km, and all distances
    # in km are distances from there
    
    a = 6378.1370 
    b = 6356.7523 
    lat = array.coords['YC'].values
    lon = array.coords['XC'].values
    lat_rad = lat*np.pi/180
    lon_rad = lon*np.pi/180
    Radius = np.sqrt((((a**2)*np.cos(lat_rad))**2 + ((b**2)*np.sin(lat_rad))**2)/((a*np.cos(lat_rad))**2 + (b*np.sin(lat_rad))**2))   
    kilometers = np.zeros(len(array))
    
    if lon_0 == None:
        lon_0 = lon[0]
    
    
    for i in range(0, len(array)):
        kilometers[i] = (np.pi*Radius*np.cos(lat_rad)*(lon[i]-lon_0))/180
    
    out = array.copy()
    
    
    out.coords['XC']= kilometers
    out = out.rename({'XC': 'km'})
    
    return out
    

What happens to the spectra when land is included? How do we handle it?

In [ ]:
print (ds.Theta)
ds.Theta[0, 0].plot(vmin = -2, vmax = 25)

In [ ]:
theta_line_0_10  = ds.Theta[0, 0].sel(YC=-30, method='nearest').sel(XC = slice(0, 10))
theta_line_5_15  = ds.Theta[0, 0].sel(YC=-30, method='nearest').sel(XC = slice(5, 15))
theta_line_10_20 = ds.Theta[0, 0].sel(YC=-30, method='nearest').sel(XC = slice(10, 20))
theta_line_0_20  = ds.Theta[0, 0].sel(YC=-30, method='nearest').sel(XC = slice(0, 20))

theta_line_0_10  = degrees_to_km(theta_line_0_10.copy(), lon_0 =0)
theta_line_5_15  = degrees_to_km(theta_line_5_15.copy(), lon_0 =0)
theta_line_10_20 = degrees_to_km(theta_line_10_20.copy(), lon_0 = 0)
theta_line_0_20  = degrees_to_km(theta_line_0_20.copy(), lon_0 = 0)


theta_line_0_20_masked = theta_line_0_20.where(theta_line_0_20.values>0)


In [ ]:
#print(theta_line_0_20)
#print(theta_line_0_20_masked.values)

#Masking replaces data values with NaNs

In [ ]:
#Masked values are NaNs; fourier transform on NaNs fails.

theta_0_10 = xrft.power_spectrum(theta_line_0_10).sel(freq_km = slice(0, np.inf))
theta_5_15 = xrft.power_spectrum(theta_line_5_15).sel(freq_km = slice(0, np.inf))
theta_10_20 = xrft.power_spectrum(theta_line_10_20).sel(freq_km=slice(0, np.inf))
theta_0_20 = xrft.power_spectrum(theta_line_0_20).sel(freq_km=slice(0, np.inf))
#theta_0_20_masked = xrft.power_spectrum(theta_line_0_20_masked).sel(freq_km=slice(0, np.inf))


theta_detrend_0_10  = xrft.power_spectrum(theta_line_0_10,   detrend = 'linear').sel(freq_km=slice(0, np.inf))
theta_detrend_5_15  = xrft.power_spectrum(theta_line_5_15,   detrend = 'linear').sel(freq_km=slice(0, np.inf))
theta_detrend_10_20 = xrft.power_spectrum(theta_line_10_20,  detrend = 'linear').sel(freq_km=slice(0, np.inf))
theta_detrend_0_20  = xrft.power_spectrum(theta_line_0_20,   detrend = 'linear').sel(freq_km=slice(0, np.inf))
#theta_detrend_0_20_masked  = xrft.power_spectrum(theta_line_0_20_masked,   detrend = 'linear').sel(freq_km=slice(0, np.inf))


theta_window_0_10 = xrft.power_spectrum(theta_line_0_10, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))
theta_window_5_15 = xrft.power_spectrum(theta_line_5_15, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))
theta_window_10_20 = xrft.power_spectrum(theta_line_10_20, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))
theta_window_0_20 = xrft.power_spectrum(theta_line_0_20, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))
#theta_window_0_20_masked = xrft.power_spectrum(theta_line_0_20_masked, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))




In [ ]:
print(len(theta_line_0_10.km.values))
print(len(theta_line_0_10.values))


x = theta_line_0_10.km.values
y = theta_line_0_10.values
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x,y)
print(slope, intercept)

In [ ]:
theta_line_0_10.plot()

x = theta_line_0_10.km.values
y = theta_line_0_10.values

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x,y)

fig = plt.figure()
ax = fig.add_subplot(111)
#ax.plot(x, y)
#ax.plot(x, slope*x+intercept)
ax.plot(x, y - (slope*x+intercept))
plt.show()


plot_spectrum_loglog(theta_0_10)




plot_spectrum_loglog(theta_detrend_0_10)

plot_spectrum_loglog(theta_window_0_10)

In [ ]:
theta_line_5_15.plot()



ax.set_ylim(15, 16.5)


plot_spectrum_loglog(theta_5_15)

plot_spectrum_loglog(theta_detrend_5_15)

plot_spectrum_loglog(theta_window_5_15)

In [ ]:
theta_line_10_20.values

In [ ]:
fig, ax = plt.subplots()
theta_line_10_20.plot(ax=ax)
ax.set_ylim(15, 16.5)



plot_spectrum_loglog(theta_10_20) 
plot_spectrum_loglog(theta_detrend_10_20)
plot_spectrum_loglog(theta_window_10_20)

In [ ]:

fig, ax = plt.subplots()
theta_line_0_20.plot(ax = ax)
ax.set_ylim(15, 16.5)

plot_spectrum_loglog(theta_0_20)
plot_spectrum_loglog(theta_detrend_0_20)
plot_spectrum_loglog(theta_window_0_20)


    

In [ ]:






#plot_spectrum_loglog(theta_window_10_20)


In [ ]:
# Futile attempts to insert multiple coordinate axes (one for each data point in latitude
# Seems unlikely to work


theta_chunk = ds.Theta[0, 0].isel(YC = slice(5, 10)).sel(XC = slice(0, 5))
theta_chunk.shape

def degree_to_km2(array):
    # Takes in DataArray as argument. Array must 
    
    a = 6378.1370 
    b = 6356.7523 
    
    lon = array.coords['XC'].values
    lon_rad = lon*np.pi/180

    for i in range(0, len(array.coords['YC'].values))

        lat = array[i].coords['YC'].values
        lat_rad = lat*np.pi/180
        Radius = np.sqrt((((a**2)*np.cos(lat_rad))**2 + ((b**2)*np.sin(lat_rad))**2)/((a*np.cos(lat_rad))**2 + (b*np.sin(lat_rad))**2))   
        meters = np.zeros(len(array))

    
    
    for i in range(0, len(array)):
        meters[i] = (np.pi*Radius*np.cos(lat_rad)*(lon[i]-lon[0]))/180
    
    out = array.copy()
    
    
    out.coords['XC']= meters
    out = out.rename({'XC': 'km'})
    
    return meters, array



In [ ]:
w_line_0_10  = ds.W[0, 0].sel(YC=-30, method='nearest').sel(XC = slice(0, 10))
w_line_5_15  = ds.W[0, 0].sel(YC=-30, method='nearest').sel(XC = slice(5, 15))
w_line_10_20 = ds.W[0, 0].sel(YC=-30, method='nearest').sel(XC = slice(10, 20))
w_line_0_20 = ds.W[0, 0].sel(YC=-30, method='nearest').sel(XC = slice(0, 20))

w_line_0_10= degrees_to_km(w_line_0_10.copy())
w_line_5_15 = degrees_to_km(w_line_5_15.copy())
w_line_10_20 = degrees_to_km(w_line_10_20.copy())
w_line_0_20 = degrees_to_km(w_line_0_20.copy())


In [ ]:

w_0_10 = xrft.power_spectrum(w_line_0_10 ).sel(freq_km = slice(0, np.inf))
w_5_15 = xrft.power_spectrum(w_line_5_15 ).sel(freq_km = slice(0, np.inf))
w_10_20 = xrft.power_spectrum(w_line_10_20).sel(freq_km=slice(0, np.inf))
w_0_20 = xrft.power_spectrum(w_line_0_20 ).sel(freq_km=slice(0, np.inf))


w_detrend_0_10 = xrft.power_spectrum(w_line_0_10, detrend = 'linear').sel(freq_km=slice(0, np.inf))
w_detrend_5_15 = xrft.power_spectrum(w_line_5_15, detrend = 'linear').sel(freq_km=slice(0, np.inf))
w_detrend_10_20 = xrft.power_spectrum(w_line_10_20, detrend = 'linear').sel(freq_km=slice(0, np.inf))
w_detrend_0_20 = xrft.power_spectrum(w_line_0_20, detrend = 'linear').sel(freq_km=slice(0, np.inf))

w_window_0_10 = xrft.power_spectrum(w_line_0_10, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))
w_window_5_15 = xrft.power_spectrum(w_line_5_15, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))
w_window_10_20 = xrft.power_spectrum(w_line_10_20, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))
w_window_0_20 = xrft.power_spectrum(w_line_0_20, detrend = 'linear', window = True).sel(freq_km=slice(0, np.inf))




In [ ]:
plot_spectrum_loglog(w_detrend_0_10)
plot_spectrum_loglog(w_detrend_5_15)
plot_spectrum_loglog(w_detrend_10_20)
plot_spectrum_loglog(w_detrend_0_20)

In [ ]:
plot_spectrum_loglog(w_window_0_10)
plot_spectrum_loglog(w_window_5_15)
plot_spectrum_loglog(w_window_10_20)
plot_spectrum_loglog(w_window_0_20)

In [ ]:
cs_0_10 = xrft.cross_spectrum(w_line_0_10, theta_line_0_10).sel(freq_km = slice(0, np.infty))
cs_0_10_detrend = xrft.cross_spectrum(w_line_0_10, theta_line_0_10, detrend = 'linear').sel(freq_km = slice(0, np.infty))
cs_0_10_window = xrft.cross_spectrum(w_line_0_10, theta_line_0_10, detrend = 'linear', window = True).sel(freq_km = slice(0, np.infty))



$$da1' = da1 - \overline{da1}; da2' = da2 - \overline{da2}$$
$$cs = \mathbb{F}(da1') * {\mathbb{F}(da2')}^*$$

In [ ]:
plot_spectrum_loglog(cs_0_10)
plot_spectrum_loglog(cs_0_10_detrend)
plot_spectrum_loglog(cs_0_10_window)

In [ ]:
# It is not clear to me what xrft.cross_spectrum() is computing. 

w_line_0_10_var = w_line_0_10 - np.mean(w_line_0_10.values)
theta_line_0_10_var = theta_line_0_10 - np.mean(theta_line_0_10.values)

option1 = np.conj(xrft.dft(w_line_0_10_var).sel(freq_km = slice(0, np.infty)))*xrft.dft(theta_line_0_10_var).sel(freq_km =slice(0, np.infty))

option3 = np.convolve(xrft.dft(w_line_0_10_var).sel(freq_km = slice(0, np.infty)),xrft.dft(theta_line_0_10_var).sel(freq_km =slice(0, np.infty))) 
                      
                      
option2 = xrft.cross_spectrum(w_line_0_10_var, theta_line_0_10_var).sel(freq_km = slice(0, np.inf))